In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point, LineString
# import itertools
from scipy.interpolate import InterpolatedUnivariateSpline

from load_data import get_pos, get_spikes
from maze_functions import find_zones, trajectory_fields
from plotting_functions import plot_intersects, plot_zone
from decode_functions import get_edges

import vdmlab as vdm

In [ ]:
xy = np.array([[2, 7],
               [4, 5],
               [6, 3],
               [8, 1],
               [2, 4]])
time = np.array([0., 1., 2., 3., 4.])
position = vdm.Position(xy, time)

In [ ]:
plt.plot(position.x, position.y, 'b.', ms=10)
plt.xlim(0.5, 8.5)
plt.ylim(0.5, 7.5)
plt.show()

In [ ]:
spikes = [vdm.SpikeTrain(np.array([3.6, 3.9])), 
          vdm.SpikeTrain(np.array([0., 0., 2.])),
          vdm.SpikeTrain(np.array([2., 2.4]))]

In [ ]:
binsize = 2
xedges = np.arange(position.x.min(), position.x.max()+binsize, binsize)
yedges = np.arange(position.y.min(), position.y.max()+binsize, binsize)

tuning_curves = vdm.tuning_curve_2d(position, spikes, xedges, yedges, sampling_rate=1.)

In [ ]:
plt.figure()
xx, yy = np.meshgrid(xedges, yedges)
for tuning_curve in tuning_curves:
    pp = plt.pcolormesh(xx, yy, tuning_curve, cmap='YlGn')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()

In [ ]:
counts_binsize = 0.5

time_edges = get_edges(position, counts_binsize, lastbin=True)
counts = vdm.get_counts(spikes, time_edges, apply_filter=False)

In [ ]:
print(time_edges)

In [ ]:
print(counts)

In [ ]:
decoding_tc = []
for tuning_curve in tuning_curves:
    decoding_tc.append(np.ravel(tuning_curve))
decoding_tc = np.array(decoding_tc)

In [ ]:
shape = tuning_curves[0].shape

In [ ]:
tuning_curves

In [ ]:
decoding_tc

In [ ]:
likelihood = vdm.bayesian_prob(counts, decoding_tc, counts_binsize)

In [ ]:
likelihood

In [ ]:
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = vdm.cartesian(xcenters, ycenters)

time_centers = (time_edges[1:] + time_edges[:-1]) / 2.

In [ ]:
time_centers

In [ ]:
decoded = vdm.decode_location(likelihood, xy_centers, time_centers)

In [ ]:
decoded.x, decoded.y, decoded.time

In [ ]:
nan_idx = np.logical_and(np.isnan(decoded.x), np.isnan(decoded.y))
decoded = decoded[~nan_idx]

In [ ]:
decoded.x, decoded.y, decoded.time

In [ ]:
x_spline = InterpolatedUnivariateSpline(position.time, position.x)
y_spline = InterpolatedUnivariateSpline(position.time, position.y)
actual_position = vdm.Position(np.hstack((x_spline(time_centers)[..., np.newaxis],
                                         (y_spline(time_centers)[..., np.newaxis]))), time_centers)

In [ ]:
actual_position.x, actual_position.y, actual_position.time

In [ ]:
error = np.abs(decoded.data - actual_position.data)

In [ ]:
avg_error = np.nanmean(error)
avg_error

## How does the 1D decoding work?

In [ ]:
x = np.array([2, 4, 6, 8, 3])
time = np.array([0., 1., 2., 3., 4.])
position = vdm.Position(x, time)

In [ ]:
spikes = [vdm.SpikeTrain(np.array([3.6, 3.9])), vdm.SpikeTrain(np.array([2.2, 2.43]))]

In [ ]:
pos_binsize = 1
tuning_curves = vdm.tuning_curve(position, spikes, binsize=pos_binsize, sampling_rate=1., gaussian_std=None)

In [ ]:
tuning_curves

In [ ]:
plt.plot(tuning_curves[0], 'b')
plt.plot(tuning_curves[1], 'm')
plt.show()

In [ ]:
counts_binsize = 0.5
time_edges = get_edges(position, counts_binsize, lastbin=True)
counts = vdm.get_counts(spikes, time_edges)

In [ ]:
time_edges.shape

In [ ]:
counts

In [ ]:
likelihood = vdm.bayesian_prob(counts, tuning_curves, counts_binsize)

In [ ]:
likelihood

In [ ]:
pos_edges = vdm.binned_position(position, pos_binsize)
x_centers = (pos_edges[1:] + pos_edges[:-1]) / 2.
x_centers = x_centers[..., np.newaxis]

time_centers = (time_edges[1:] + time_edges[:-1]) / 2.

decoded = vdm.decode_location(likelihood, x_centers, time_centers)
decoded

In [ ]:
nan_idx = np.isnan(decoded.x)
decoded = decoded[~nan_idx]

In [ ]:
decoded.x, decoded.time

In [ ]:
spline = InterpolatedUnivariateSpline(position.time, position.x)
actual_position = vdm.Position(spline(decoded.time), decoded.time)

In [ ]:
actual_position.x, actual_position.time

In [ ]:
error = np.abs(decoded.x - actual_position.x)

In [ ]:
decoded.x, actual_position.x

In [ ]:
avg_error = np.mean(error)
avg_error

# check velocity 1D

In [ ]:
x = np.array([2, 4, 6, 8, 3])
time = np.array([0., 1., 2., 3., 4.])
position = vdm.Position(x, time)

spikes = [vdm.SpikeTrain(np.array([3.6, 3.9])), 
          vdm.SpikeTrain(np.array([2.2, 2.4])),
          vdm.SpikeTrain(np.array([0.6, 0.9])),
          vdm.SpikeTrain(np.array([1., 1.1])), 
          vdm.SpikeTrain(np.array([1.7, 1.9]))]

pos_binsize = 1
tuning_curves = vdm.tuning_curve(position, spikes, binsize=pos_binsize, sampling_rate=1., gaussian_std=None)

counts_binsize = 0.5
time_edges = get_edges(position, counts_binsize, lastbin=True)
counts = vdm.get_counts(spikes, time_edges)

likelihood = vdm.bayesian_prob(counts, tuning_curves, counts_binsize)

pos_edges = vdm.binned_position(position, pos_binsize)
x_centers = (pos_edges[1:] + pos_edges[:-1]) / 2.
x_centers = x_centers[..., np.newaxis]

time_centers = (time_edges[1:] + time_edges[:-1]) / 2.

decoded = vdm.decode_location(likelihood, x_centers, time_centers)

nan_idx = np.isnan(decoded.x)
decoded = decoded[~nan_idx]

print(decoded.x, decoded.time)

In [ ]:
decode_jumps = vdm.remove_teleports(decoded, speed_thresh=0, min_length=1)

In [ ]:
decode_jumps.time, decode_jumps.x

## Counts filtering parameter check

In [ ]:
std = [0.1, 0.025, 0.01, 0.002, None, 0.5, 1.0]
error = [29.9113296726, 29.3032199091, 45.8427697608, 45.8427697608, 45.8427697608, 28.623598705, 29.0339763118]

In [ ]:
plt.plot(std, error, '.', ms=15)
plt.xlim(-0.1, 1.1)
plt.ylim(25, 48)
plt.show()

## Other stuff

In [ ]:
from load_data import get_pos, get_spikes, get_lfp

import info.R063d2_info as r063d2
import info.R063d3_info as r063d3

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from shapely.geometry import Point, LineString
import random
import seaborn as sns

import vdmlab as vdm

from load_data import get_pos, get_spikes
from maze_functions import find_zones
from tuning_curves_functions import get_tc_1d, find_ideal
from decode_functions import get_edges, point_in_zones, compare_rates
from plotting_functions import (plot_decoded, plot_decoded_pause, plot_decoded_errors,
                                plot_compare_decoded_pauses)

In [ ]:
# pickle_filepath = 'C:\\Users\\Emily\\Code\\emi_shortcut\\cache\\pickled\\'
# output_filepath = 'C:\\Users\\Emily\\Code\\emi_shortcut\\plots\\'
pickle_filepath = 'E:\\code\\emi_shortcut\\cache\\pickled\\'
output_filepath = 'E:\\code\\emi_shortcut\\plots\\'

In [ ]:
info = r063d2

In [ ]:
u_length = LineString(info.u_trajectory).length
shortcut_length = LineString(info.shortcut_trajectory).length
novel_length = LineString(info.novel_trajectory).length
print(u_length, shortcut_length, novel_length)

In [ ]:
def get_decoded(infos, experiment_time='tracks', shuffle_id=False):
    """Finds combined decoded from all sessions.

    Parameters
    ----------
    info: list of modules
    experiment_time: str
    shuffle_id: bool
        Defaults to False (not shuffled)

    Returns
    -------
    combined_decoded: dict of vdmlab.Position objects
        With u, shortcut, novel, other, together as keys.
    combined_errors: list of np.arrays
    total_times: list

    """
    total_times = []
    combined_errors = []
    combined_lengths = dict(u=[], shortcut=[], novel=[], other=[], together=[])
    combined_decoded = dict(u=[], shortcut=[], novel=[], other=[], together=[])

    for info in infos:
        print(info.session_id)
        position = get_pos(info.pos_mat, info.pxl_to_cm)
        spikes = get_spikes(info.spike_mat)

        speed = position.speed(t_smooth=0.5)
        run_idx = np.squeeze(speed.data) >= 0.1
        run_pos = position[run_idx]

        track_starts = [info.task_times['phase1'].start,
                        info.task_times['phase2'].start,
                        info.task_times['phase3'].start]
        track_stops = [info.task_times['phase1'].stop,
                       info.task_times['phase2'].stop,
                       info.task_times['phase3'].stop]

        # track_start = info.task_times['phase3'].start
        # track_stop = info.task_times['phase3'].stop

        track_pos = run_pos.time_slices(track_starts, track_stops)

        track_spikes = [spiketrain.time_slices(track_starts, track_stops) for spiketrain in spikes]

        binsize = 3
        xedges = np.arange(track_pos.x.min(), track_pos.x.max() + binsize, binsize)
        yedges = np.arange(track_pos.y.min(), track_pos.y.max() + binsize, binsize)

        tuning_curves = vdm.tuning_curve_2d(track_pos, track_spikes, xedges, yedges, gaussian_sigma=0.1)

        if shuffle_id:
            random.shuffle(tuning_curves)

        if experiment_time == 'tracks':
            decode_spikes = track_spikes
        else:
            decode_spikes = [spiketrain.time_slice(info.task_times[experiment_time].start,
                                                   info.task_times[experiment_time].stop) for spiketrain in spikes]

        counts_binsize = 0.025
        time_edges = get_edges(run_pos, counts_binsize, lastbin=True)
        counts = vdm.get_counts(decode_spikes, time_edges, gaussian_std=0.025)

        decoding_tc = []
        for tuning_curve in tuning_curves:
            decoding_tc.append(np.ravel(tuning_curve))
        decoding_tc = np.array(decoding_tc)

        likelihood = vdm.bayesian_prob(counts, decoding_tc, counts_binsize)

        xcenters = (xedges[1:] + xedges[:-1]) / 2.
        ycenters = (yedges[1:] + yedges[:-1]) / 2.
        xy_centers = vdm.cartesian(xcenters, ycenters)

        time_centers = (time_edges[1:] + time_edges[:-1]) / 2.

        decoded = vdm.decode_location(likelihood, xy_centers, time_centers)
        nan_idx = np.logical_and(np.isnan(decoded.x), np.isnan(decoded.y))
        decoded = decoded[~nan_idx]

        if not decoded.isempty:
            decoded = vdm.remove_teleports(decoded, speed_thresh=10, min_length=3)

        actual_x = np.interp(decoded.time, track_pos.time, track_pos.x)
        actual_y = np.interp(decoded.time, track_pos.time, track_pos.y)

        actual_position = vdm.Position(np.hstack((actual_x[..., np.newaxis], actual_y[..., np.newaxis])), decoded.time)

        errors = actual_position.distance(decoded)

        zones = find_zones(info, expand_by=7)
        actual_zones = point_in_zones(actual_position, zones)
        decoded_zones = point_in_zones(decoded, zones)

        total_times.append(len(time_edges) - 1)

        combined_errors.extend(errors)

        combined_lengths['u'].append(LineString(info.u_trajectory).length)
        combined_lengths['shortcut'].append(LineString(info.shortcut_trajectory).length)
        combined_lengths['novel'].append(LineString(info.novel_trajectory).length)

        # combined_actual['u'].append(actual_zones['u'])
        # combined_actual['shortcut'].append(actual_zones['shortcut'])
        # combined_actual['novel'].append(actual_zones['novel'])
        # combined_actual['other'].append(actual_zones['other'])
        # combined_actual['together'].append(len(actual_zones['u'].time) + len(actual_zones['shortcut'].time) +
        #                                    len(actual_zones['novel'].time) + len(actual_zones['other'].time))

        combined_decoded['u'].append(decoded_zones['u'])
        combined_decoded['shortcut'].append(decoded_zones['shortcut'])
        combined_decoded['novel'].append(decoded_zones['novel'])
        combined_decoded['other'].append(decoded_zones['other'])
        combined_decoded['together'].append(len(decoded_zones['u'].time) + len(decoded_zones['shortcut'].time) +
                                            len(decoded_zones['novel'].time) + len(decoded_zones['other'].time))

    return combined_decoded, combined_errors, total_times, combined_lengths


# def compare_decoded_actual(combined_actual, combined_decoded, shuffle_id, output_filepath, pause=None):
#     keys = ['u', 'shortcut', 'novel']
#
#     actual = dict(u=[], shortcut=[], novel=[])
#     decode = dict(u=[], shortcut=[], novel=[])
#
#     n_sessions = len(combined_actual['together'])
#
#     # length_tracks = info.track_length['u'] + info.track_length['shortcut'] + info.track_length['novel']
#
#     for key in keys:
#         if len(combined_actual['together']) != len(combined_decoded['together']):
#             raise ValueError("must have same number of decoded and actual samples")
#
#         for val in range(n_sessions):
#             actual[key].append(len(combined_actual[key][val].time)/combined_actual['together'][val])
#             decode[key].append(len(combined_decoded[key][val].time)/combined_decoded['together'][val])
#
#     if shuffle_id and pause is None:
#         filename = 'combined-phase3-id_shuffle_decoded.png'
#     elif shuffle_id and pause is not None:
#         filename = 'combined-' + pause + '-id_shuffle_decoded.png'
#     elif pause is not None and not shuffle_id:
#         filename = 'combined-' + pause + '_decoded.png'
#     elif not shuffle_id and pause is None:
#         filename = 'combined-phase3_decoded.png'
#     else:
#         filename = 'unknown_combination.png'
#     savepath = os.path.join(output_filepath, filename)
#
#     if pause is not None:
#         plot_compare_decoded_track(decode, max_y=1.0, savepath=savepath)
#     else:
#         plot_compare_decoded_track(decode, actual, distance=str(round(np.mean(combined_errors), 2)),
#                                    max_y=1.0, savepath=savepath)
#
#     print('Decoded errors:', combined_errors)


def normalized_time_spent(combined_decoded, n_sessions, lengths):
    decoded_linger = dict(u=[], shortcut=[], novel=[])
    decoded_length = dict(u=[], shortcut=[], novel=[])
    for val in range(n_sessions):
        decode = dict()
        length = dict()
        for key in decoded_linger:
            decode[key] = combined_decoded[key][val]
            length[key] = lengths[key][val]
        norm_decoded = compare_rates(decode)
        len_decoded = compare_lengths(decode, length)
        for key in decoded_linger:
            decoded_linger[key].append(norm_decoded[key])
            decoded_length[key].append(len_decoded[key])

    filename = 'combined-time-norm_tracks_decoded.png'
    savepath = os.path.join(output_filepath, filename)
    y_label = 'Points normalized by time spent'
    plot_decoded(decoded_linger, y_label=y_label, max_y=None, savepath=savepath)

    filename = 'combined-length-norm_tracks_decoded.png'
    savepath = os.path.join(output_filepath, filename)
    y_label = 'Points normalized by track length'
    plot_decoded(decoded_length, y_label=y_label, max_y=None, savepath=savepath)


infos = [r063d2, r063d3]
# infos = [r063d2, r063d3, r063d4, r063d5, r063d6,
#          r066d1, r066d2, r066d3, r066d4, r066d5,
#          r067d1, r067d2, r067d3, r067d4, r067d5,
#          r068d1, r068d2, r068d3, r068d4, r068d5]


# Plot decoding errors during track times
if 0:
    combined_decoded, combined_errors, total_times, lengths = get_decoded(infos, shuffle_id=False)
    print('Mean decoded errors:', np.mean(combined_errors))
    shuffled_decoded, shuffled_errors, shuffled_times, shuffled_lengths = get_decoded(infos, shuffle_id=True)
    print('Mean shuffled errors:', np.mean(combined_errors))
    filename = 'combined-errors_decoded.png'
    savepath = os.path.join(output_filepath, filename)
    plot_decoded_errors(combined_errors, shuffled_errors, savepath=savepath)

# Plot proportion of pauseA and pauseB spent in each trajectory
if 1:
    experiment_time = 'pauseA'
    decoded_pausea, errors_pausea, times_pausea, lengths_pausea = get_decoded(infos, experiment_time, shuffle_id=False)
    filename = 'combined-' + experiment_time + '_decoded.png'
    savepath = os.path.join(output_filepath, filename)
    plot_decoded_pause(decoded_pausea, times_pausea, savepath=savepath)

    experiment_time = 'pauseB'
    decoded_pauseb, errors_pauseb, times_pauseb, lengths_pauseb = get_decoded(infos, experiment_time, shuffle_id=False)
    filename = 'combined-' + experiment_time + '_decoded.png'
    savepath = os.path.join(output_filepath, filename)
    plot_decoded_pause(decoded_pauseb, times_pauseb, savepath=savepath)

    filename = 'combined-pauses_decoded.png'
    savepath = os.path.join(output_filepath, filename)
    plot_compare_decoded_pauses(decoded_pausea, times_pausea, decoded_pauseb, times_pauseb, ['Pause A', 'Pause B'],
                                savepath=False)

In [ ]:
def plot_compare_decoded_pauses(decoded_1, times_1, decoded_2, times_2, labels, savepath=None):
    """Plots barplot comparing decoded during two phases

    Parameters
    ----------
    decoded_1: dict
        With u, shortcut, novel, other as keys, each a vdmlab.Position object.
    times_1: list of floats
    decoded_2: dict
        With u, shortcut, novel, other as keys, each a vdmlab.Position object.
    times_2: list of floats
    labels: list of str
    savepath : str or None
        Location and filename for the saved plot.

    """
    decode_1 = dict(u=[], shortcut=[], novel=[])
    for key in decode_1:
        for session in range(len(times_1)):
            decode_1[key].append(len(decoded_1[key][session].time)/times_1[session])

    u_dict1 = dict(total=decode_1['u'], trajectory='U', exp_time=labels[0])
    shortcut_dict1 = dict(total=decode_1['shortcut'], trajectory='Shortcut', exp_time=labels[0])
    novel_dict1 = dict(total=decode_1['novel'], trajectory='Novel', exp_time=labels[0])

    u1 = pd.DataFrame(u_dict1)
    shortcut1 = pd.DataFrame(shortcut_dict1)
    novel1 = pd.DataFrame(novel_dict1)

    decode_2 = dict(u=[], shortcut=[], novel=[])
    for key in decode_2:
        for session in range(len(times_2)):
            decode_2[key].append(len(decoded_2[key][session].time)/times_2[session])

    u_dict2 = dict(total=decode_2['u'], trajectory='U', exp_time=labels[1])
    shortcut_dict2 = dict(total=decode_2['shortcut'], trajectory='Shortcut', exp_time=labels[1])
    novel_dict2 = dict(total=decode_2['novel'], trajectory='Novel', exp_time=labels[1])

    u2 = pd.DataFrame(u_dict2)
    shortcut2 = pd.DataFrame(shortcut_dict2)
    novel2 = pd.DataFrame(novel_dict2)
    data = pd.concat([u1, shortcut1, novel1, u2, shortcut2, novel2])

    together = data.groupby(['trajectory', 'exp_time'])['total'].mean()

    def plot_v3(data, exp_labels):
        fig = sns.FacetGrid(data, col='trajectory', col_order=['U', 'Shortcut', 'Novel'], sharex=False)

        fig.map(sns.barplot, 'trajectory', 'total', 'exp_time',  hue_order=[exp_labels[0], exp_labels[1]])
        axes = np.array(fig.axes.flat)

        return plt.gcf(), axes

    def set_labels(fig, axes, exp_labels):
        labels = ['U', 'Shortcut', 'Novel']

        for i, ax in enumerate(axes):
            ax.set_xticks([-.2, .2])
            ax.set_xticklabels([exp_labels[0], exp_labels[1]])
            ax.set_xlabel(labels[i])
            ax.set_ylabel('')
            ax.set_title('')

        axes.flat[0].set_ylabel('Proportion of time')

        sns.despine(ax=axes[1], left=True)
        sns.despine(ax=axes[2], left=True)

    def set_style():
        plt.style.use(['seaborn-white', 'seaborn-paper'])

    def color_bars(axes):
        colors = sns.color_palette('colorblind')
        edges = sns.color_palette('deep')
        for i in range(3):
            p1, p2 = axes[i].patches

            p1.set_color(colors[i])

            p2.set_color(colors[i])
            p2.set_edgecolor(edges[i])
            p2.set_hatch('///')

    def set_size(fig):
        fig.set_size_inches(6, 3)
        plt.tight_layout()

    set_style()
    fig, axes = plot_v3(data, labels)
    set_labels(fig, axes, labels)
    color_bars(axes)

    if savepath:
        fig.savefig(savepath)

In [ ]:
plot_compare_decoded_pauses(decoded_pausea, times_pausea, decoded_pauseb, times_pauseb, 
                            ['Pause A', 'Pause B'], savepath=False)

In [ ]:
import pandas as pd

In [ ]:
decoded_1 = decoded_pausea
times_1 = times_pausea
decoded_2 = decoded_pauseb
times_2 = times_pauseb
labels = ['Pause A', 'Pause B']

decode_1 = dict(u=[], shortcut=[], novel=[])
for key in decode_1:
    for session in range(len(times_1)):
        decode_1[key].append(len(decoded_1[key][session].time)/times_1[session])

u_dict1 = dict(total=decode_1['u'], trajectory='U', exp_time=labels[0])
shortcut_dict1 = dict(total=decode_1['shortcut'], trajectory='Shortcut', exp_time=labels[0])
novel_dict1 = dict(total=decode_1['novel'], trajectory='Novel', exp_time=labels[0])

u1 = pd.DataFrame(u_dict1)
shortcut1 = pd.DataFrame(shortcut_dict1)
novel1 = pd.DataFrame(novel_dict1)

decode_2 = dict(u=[], shortcut=[], novel=[])
for key in decode_2:
    for session in range(len(times_2)):
        decode_2[key].append(len(decoded_2[key][session].time)/times_2[session])

u_dict2 = dict(total=decode_2['u'], trajectory='U', exp_time=labels[1])
shortcut_dict2 = dict(total=decode_2['shortcut'], trajectory='Shortcut', exp_time=labels[1])
novel_dict2 = dict(total=decode_2['novel'], trajectory='Novel', exp_time=labels[1])

u2 = pd.DataFrame(u_dict2)
shortcut2 = pd.DataFrame(shortcut_dict2)
novel2 = pd.DataFrame(novel_dict2)
data = pd.concat([u1, shortcut1, novel1, u2, shortcut2, novel2])

# together = data.groupby(['trajectory', 'exp_time'])['total'].mean()

In [ ]:
data

In [ ]:
def plot_v2(data, exp_labels):
    ax = sns.barplot(
        x='trajectory', y='total', hue='exp_time',
        order=['U', 'Shortcut', 'Novel'],
        hue_order=[exp_labels[0], exp_labels[1]],
        data=data)

    return plt.gcf(), ax

In [ ]:
def plot_v3(data, exp_labels):
    g = sns.FacetGrid(
        data,
        col='trajectory',
        col_order=['U', 'Shortcut', 'Novel'],
        sharex=False)

    g.map(sns.barplot,
        'trajectory', 'total', 'exp_time',
        hue_order=[exp_labels[0], exp_labels[1]])

    axes = np.array(g.axes.flat)

    return plt.gcf(), axes

def set_labels(fig, axes, exp_labels):
    labels = ['U', 'Shortcut', 'Novel']

    for i, ax in enumerate(axes):
        ax.set_xticks([-.2, .2])
        ax.set_xticklabels([exp_labels[0], exp_labels[1]])
        ax.set_xlabel(labels[i])
        ax.set_ylabel('')
        ax.set_title('')

    axes.flat[0].set_ylabel('Proportion of time')
    
    sns.despine(ax=axes[1], left=True)
    sns.despine(ax=axes[2], left=True)
    
def set_style():
    plt.style.use(['seaborn-white', 'seaborn-paper'])

def get_colors():
    return np.array([
        [0.1, 0.1, 0.1],           # black
        [0.4, 0.4, 0.4],           # very dark gray
        [0.984375, 0.7265625, 0],  # dark yellow
        [1, 1, 0.9],               # light yellow
        [0.7, 0.7, 0.7],           # dark gray
        [0.9, 0.9, 0.9],           # light gray
        ])

def color_bars(axes, colors):
    for i in range(3):
        dark_color = colors[i*2]
        light_color = colors[i*2 + 1]

        p1, p2 = axes[i].patches

        p1.set_color(dark_color)

        p2.set_color(light_color)
        p2.set_edgecolor(dark_color)
        p2.set_hatch('////')

def set_size(fig):
    fig.set_size_inches(6, 3)
    plt.tight_layout()

In [ ]:
set_style()
fig, axes = plot_v3(data, labels)
set_labels(fig, axes, labels)
color_bars(axes, get_colors())

plt.show()

## Other

In [ ]:
pos = np.random.rand(20, 2)

In [ ]:
pos

In [ ]:
split_idx = np.array([4, 6, 10])

In [ ]:
all_idx = [idx for idx in np.split(np.arange(pos.shape[0]), split_idx) if idx.size > 3]

In [ ]:
pos[np.hstack(all_idx)]

In [ ]:
time = np.linspace(0, np.pi*2, 201)
data = np.hstack((np.sin(time)))

In [ ]:
plt.plot(time, data, '.')
plt.show()

In [ ]:
position = vdm.Position(data, time)

In [ ]:
speed = position.speed()

In [ ]:
plt.plot(speed.time, speed.data)
plt.show()

In [ ]:
run_idx = np.squeeze(speed.data) >= 0.5

In [ ]:
run_idx

In [ ]:
position = vdm.Position(data, time)
speed = position.speed()
run_idx = np.squeeze(speed.data) >= 0.7
run_position = position[run_idx]

len(run_position.x)

In [ ]:
assert np.allclose(len(run_position.x), 100)

In [ ]:
velocity = self[1:].distance(self[:-1])
velocity /= np.diff(self.time)
velocity = np.hstack(([0], velocity))

if t_smooth is not None:
    dt = np.median(np.diff(self.time))
    filter_length = np.ceil(t_smooth / dt)
    velocity = np.convolve(velocity, np.ones(int(filter_length))/filter_length, 'same')

return AnalogSignal(velocity, self.time)

In [ ]:
t_smooth=0.5
velocity = np.diff(np.squeeze(position.data))
velocity /= np.diff(position.time)
velocity = np.hstack(([0], velocity))

dt = np.median(np.diff(position.time))
filter_length = np.ceil(t_smooth / dt)
velocity = np.convolve(velocity, np.ones(int(filter_length))/filter_length, 'same')

In [ ]:
velocity

In [ ]:
a = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
a

In [ ]:
np.hstack((a[0:2], a[6:8]))

In [ ]:
indices = []
for t_start, t_stop in zip(np.array([0, 6]), np.array([2, 8])):
    indices.append((a >= t_start) & (a <= t_stop))
indices = np.any(np.column_stack((indices)),axis=1)
a[indices]

In [ ]:
res = np.any(np.column_stack((indices)),axis=1)
res

In [ ]:
spikes = [vdm.SpikeTrain(np.array([0., 0., 1.])), vdm.SpikeTrain(np.array([3.6, 3.9]))]

In [ ]:
one_line = LineString([[2, 0], [2, 2], [2, 4], [2, 6], [2, 8], [2, 10]])

one_start = Point([2, 2])
one_stop = Point([2, 8])

expand_by = 1

one_zone = vdm.expand_line(one_start, one_stop, one_line, expand_by)

In [ ]:
this_idx = []
for pos_idx in range(len(position.time)):
    point = Point([position.x[pos_idx], position.y[pos_idx]])
    if one_zone.contains(point):
        this_idx.append(pos_idx)
        
this_pos = position[this_idx]
linear = this_pos.linearize(one_line, one_zone)

In [ ]:
linear.x

In [ ]:
plt.plot(position.x, position.y, 'g.', ms=10)
# plt.plot([2, 4, 2], [7, 5, 4], 'm.', ms=20)
plt.plot([2, 6], [4, 3], 'm.', ms=20)
plt.plot(one_zone.exterior.xy[0], one_zone.exterior.xy[1], 'b', lw=1)
plt.xlim(-1, 10)
plt.ylim(-1, 10)
plt.show()

In [ ]:
def expand_line(start_pt, stop_pt, line, expand_by):
    line_expanded = line.buffer(expand_by)
    zone = start_pt.union(line_expanded).union(stop_pt)
    
    return zone

def find_zones(info, expand_by=6):
    u_line = LineString(info.u_trajectory)
    shortcut_line = LineString(info.shortcut_trajectory)
    novel_line = LineString(info.novel_trajectory)

    u_start = Point(info.u_trajectory[0])
    u_stop = Point(info.u_trajectory[-1])
    shortcut_start = Point(info.shortcut_trajectory[0])
    shortcut_stop = Point(info.shortcut_trajectory[-1])
    novel_start = Point(info.novel_trajectory[0])
    novel_stop = Point(info.novel_trajectory[-1])
    pedestal_center = Point(info.path_pts['pedestal'][0], info.path_pts['pedestal'][1])
    pedestal = pedestal_center.buffer(expand_by*2.2)

    zone = dict()
    zone['u'] = expand_line(u_start, u_stop, u_line, expand_by)
    zone['shortcut'] = expand_line(shortcut_start, shortcut_stop, shortcut_line, expand_by)
    zone['novel'] = expand_line(novel_start, novel_stop, novel_line, expand_by)
    zone['ushort'] = zone['u'].intersection(zone['shortcut'])
    zone['unovel'] = zone['u'].intersection(zone['novel'])
    zone['uped'] = zone['u'].intersection(pedestal)
    zone['shortped'] = zone['shortcut'].intersection(pedestal)
    zone['novelped'] = zone['novel'].intersection(pedestal)
    zone['pedestal'] = pedestal
    
    return zone

In [ ]:
def trajectory_fields(tuning_curves, zone, xedges, yedges, field_thresh):
    
    xcenters = np.array((xedges[1:] + xedges[:-1]) / 2.)
    ycenters = np.array((yedges[1:] + yedges[:-1]) / 2.)
    
    tuning_points = []
    for i in itertools.product(ycenters, xcenters):
        tuning_points.append(i)
    tuning_points = np.array(tuning_points)

    this_neuron = 0
    fields_tc = dict(u=[], shortcut=[], novel=[], pedestal=[])
    fields_neuron = dict(u=[], shortcut=[], novel=[], pedestal=[])
    for neuron_tc in tuning_curves:
        this_neuron += 1
        field_idx = neuron_tc.flatten() > field_thresh
        field = tuning_points[field_idx]
        for pt in field:
            point = Point([pt[0], pt[1]])
            if zone['u'].contains(point) or zone['ushort'].contains(point) or zone['unovel'].contains(point):
                if this_neuron not in fields_neuron['u']:
                    fields_tc['u'].append(neuron_tc)
                    fields_neuron['u'].append(this_neuron)
            if zone['shortcut'].contains(point) or zone['shortped'].contains(point):
                if this_neuron not in fields_neuron['shortcut']:
                    fields_tc['shortcut'].append(neuron_tc)
                    fields_neuron['shortcut'].append(this_neuron)
            if zone['novel'].contains(point) or zone['novelped'].contains(point):
                if this_neuron not in fields_neuron['novel']:
                    fields_tc['novel'].append(neuron_tc)
                    fields_neuron['novel'].append(this_neuron)
            if zone['pedestal'].contains(point):
                if this_neuron not in fields_neuron['pedestal']:
                    fields_tc['pedestal'].append(neuron_tc)
                    fields_neuron['pedestal'].append(this_neuron)
                
    return fields_tc

In [ ]:
import sys
# sys.path.append('C:\\Users\\Emily\\Code\\emi_shortcut\\info')
sys.path.append('E:\\code\\emi_shortcut\\info')
import info.R063d3_info as r063d3
info = r063d3

In [ ]:
pickle_filepath = 'E:\\code\\emi_shortcut\\cache\\pickled'
# pickle_filepath = 'C:\\Users\\Emily\\Code\\emi_shortcut\\cache\\pickled'

In [ ]:
position = get_pos(info.pos_mat, info.pxl_to_cm)
spikes = get_spikes(info.spike_mat)

In [ ]:
binsize = 3
xedges = np.arange(position.x.min(), position.x.max()+binsize, binsize)
yedges = np.arange(position.y.min(), position.y.max()+binsize, binsize)

speed = position.speed(t_smooth=0.5)
run_idx = np.squeeze(speed.data) >= info.run_threshold
run_pos = position[run_idx]

t_start = info.task_times['phase3'].start
t_stop = info.task_times['phase3'].stop

sliced_pos = run_pos.time_slice(t_start, t_stop)

sliced_spikes = [spiketrain.time_slice(t_start, t_stop) for spiketrain in spikes]

tuning_curves = vdm.tuning_curve_2d(sliced_pos, sliced_spikes, xedges, yedges, gaussian_sigma=0.2)

In [ ]:
zones = find_zones(info)

In [ ]:
type(zones['u'])

In [ ]:
fields_tc = trajectory_fields(tuning_curves, zones, xedges, yedges, field_thresh=5)

In [ ]:
print(len(fields_tc['novel']))

In [ ]:
tuning_curves[5]

In [ ]:
plt.figure()
xx, yy = np.meshgrid(xedges, yedges)
for tuning_curve in fields_tc['novel']:
    pp = plt.pcolormesh(xx, yy, tuning_curve, cmap='YlGn')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()